Python program to read in the sounding profile, establish a distribution of hailstones, and send them through the melting program. Considers only falling hail, melting, and terminal velocity. 

Simulations ran for each T-28 case. 

Systematically testing different up/down drafts. Some are constant, some increase linearly, and some include downdrafts. 


In [1]:
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np
import melt_G1969orig, melt_RH87, melt_WC2020, melt_ZL1994
import terminl_Bohm89X_Re, terminl_H2018V_D, terminl_H2018X_Re, terminl_HW2014X_Re, terminl_RH87X_Re
from metpy.calc import mixing_ratio_from_relative_humidity, potential_temperature, density, moist_lapse
from metpy.calc import lcl, parcel_profile, wind_components, cape_cin, pressure_to_height_std, height_to_pressure_std
from metpy.calc import relative_humidity_from_dewpoint, wet_bulb_temperature, equivalent_potential_temperature
from metpy.units import units
from metpy.plots import SkewT
import datetime
import netCDF4 as nc4
import geocat.viz as gv
from math import pi

In [2]:
# Set directories

# Directories
PlotDirectory = '/Directory/Save/Figures/'

SoundingDirectory = '/Directory/Sounding/Data/'
T28Directory = '/Directory/T28/Data/'

# Files
T28File = 'Name_of_T28_File.nc'


In [3]:
# Set parameterization names

MeltNames = ['mG1969orig', 'mRH87', 'mWC2020', 'mZL1994']
TerminalNames = ['tBohm89xre', 'tH2018vd', 'tH2018xre', 'tHW2014xre', 'tRH87xre']


In [4]:
# Set flags

# Behavior
SheddingFlag = True

HailReleaseHeight = 7.8


In [5]:
# Set hail parameters

CRIT = 2.0E-4 # mass capable of being supported on the stone's surface


In [6]:
# Set time step and number of steps

secdel = 1.0 # 1 second time step
numt = 3500 # number of times


In [8]:
# Define general height profile, used to create vertical velocity profile

HeightProfile = np.arange(0, 10.1, 0.01)*1000 # m


In [9]:
# Set up plotting

cmap_diff = matplotlib.colors.LinearSegmentedColormap.from_list("", ["indigo","darkblue","mediumblue","blue","royalblue",
                                                                     "dodgerblue","deepskyblue","lightskyblue","skyblue",
                                                                     "powderblue","lightblue","lightcyan","white",
                                                                     "mistyrose","pink","lightcoral","salmon","coral","tomato",
                                                                     "orangered","red","crimson","firebrick","brown","darkred"])

PlotColors = ['forestgreen', 'mediumblue', 'darkorange', 'crimson', 'dimgrey', 'black']
PlotLabels = ['Constant -2', 'Constant 2', 'Linear -2 -> 2', 'Linear 0 -> 8', 'Linear -2 -> 5', 'Constant 5'] 
PlotSaveNames = ['C-2', 'C2', 'L2', 'L8', 'L5', 'C5']


At each timestep during each flight:
   1) calculate terminal velocity
   2) calculate distance hailstone has fallen
   3) interpolate temp, density, pressure, and qv at new height
   5) calculate how much has melted over that distance and new diameter
   6) Repeat

In [11]:
# Load T28 data, sounding data, and run HAILCAST

# Load t28 data
FlightData = nc4.Dataset(T28Directory + T28File, 'r')
    
Bins = np.array(FlightData.variables['Bins'])[:] #m
#num_each_diameter = np.array(FlightData.variables['TotalConcentration'])[:] #m^-3
num_each_diameter = np.array(FlightData.variables['MeanConcentration'])[:] #m^-3
VerticalVelocity = np.array(FlightData.variables['VerticalVelocity'])[:] #m/s
Diameter = np.array(FlightData.variables['Diameter'])[:] #m, mass weighted mean diameter
Temperature = np.array(FlightData.variables['Temperature'])[:] #C
FlightAltitude  = np.array(FlightData.variables['Altitude'])[:] #m
FlightDay = np.nanmean(np.array(FlightData.variables['Day'][:]))
FlightMonth = np.nanmean(np.array(FlightData.variables['Month'][:]))
FlightYear = np.nanmean(np.array(FlightData.variables['Year'][:]))
FlightTimes = np.array(FlightData.variables['Times'])[:]

FlightData.close()

FullTimes = []
for t in range(0, len(FlightTimes)):
    FullTimes = np.append(FullTimes, 
                          datetime.datetime(int(FlightYear), int(FlightMonth), int(FlightDay), 0, 0, 0) + 
                          datetime.timedelta(seconds=float(FlightTimes[t])))

# Select bins only with non-zero numbers
d_0 = Bins[num_each_diameter > 0]
numd = len(d_0)

num_each_diameter = num_each_diameter[num_each_diameter > 0]

# Load sounding data
SoundingData = np.genfromtxt(DataDirectory + '/mobile1.steps.qc_soundings/D200006292355.st1QC.cls',
                             usecols=(1, 14, 2, 3, 8, 7), skip_header=14, dtype=('f','f','f','f'), 
                             names=('Pressure', 'Height', 'Temperature', 'DewPoint'))

Pressure = np.array(SoundingData['Pressure']) # hPa
Height = np.array(SoundingData['Height']) #m
Temperature = np.array(SoundingData['Temperature']) # C
DewPoint = np.array(SoundingData['DewPoint']) # C

# Omit missing data
Keep = np.squeeze(np.array(np.where(((Pressure != 9999) & ~np.isnan(Pressure) & (Temperature != 999) & (DewPoint != 999)))))
Pressure = Pressure[Keep]
Height = Height[Keep]
Temperature = Temperature[Keep]
DewPoint = DewPoint[Keep]

Keep = np.squeeze(np.array(np.where(Pressure[1::] != Pressure[0:-1])))
Pressure = Pressure[Keep]
Height = Height[Keep]
Temperature = Temperature[Keep]
DewPoint = DewPoint[Keep]

# Isolate Surface data
sfcP = Pressure[0]
sfcT = Temperature[0]
sfcD = DewPoint[0]

# Calculate environmental characteristics
RelativeHumidity = relative_humidity_from_dewpoint(Temperature*units('degC'), DewPoint*units('degC'))
        
MixingRatio = mixing_ratio_from_relative_humidity(Pressure*units('hPa'), Temperature*units('degC'), RelativeHumidity)
            
rho = density(Pressure*units('hPa'), Temperature*units('degC'), MixingRatio) # density
            
ThetaE = equivalent_potential_temperature(Pressure*units('hPa'), Temperature*units('degC'), DewPoint*units('degC'))

parcel_prof = parcel_profile(Pressure*units('hPa'), sfcT*units('degC'), sfcD*units('degC'))

# Calculate additional data points
lcl_pressure, lcl_temperature = lcl(sfcP*units('hPa'), sfcT*units('degC'), sfcD*units('degC'))

# Determine index of freezing level and lcl
lclIndex = np.squeeze(np.array(np.nanargmin(np.absolute(np.subtract(Pressure, np.ones_like(Pressure)*np.array(lcl_pressure))))))
FlightIndex = np.array(np.nanargmin(np.absolute(np.subtract(Height, np.ones_like(Height)*np.nanmean(FlightAltitude)))))

# Determine index associated with height that hail will be released
HailReleaseIndex = np.array(np.nanargmin(np.absolute(np.subtract(Height, np.ones_like(Height)*(HailReleaseHeight*1000)))))

lcl_height = Height[lclIndex] - Height[0]
#freezing_height = Height[frzlvl]

# Determine mixing ratio in cloud and add to sub cloud profile. Remember should be saturated above the LCL.
# Calculate temperature in cloud
temp_cloud = np.copy(parcel_prof[lclIndex:HailReleaseIndex+1])

# Calculate mixing ratio
satq = mixing_ratio_from_relative_humidity(Pressure[lclIndex:HailReleaseIndex+1]*units('hPa'), temp_cloud, 1.)
# mixing ratio assuming RH=1 between lcl and freezing level, merge raw mixing ratio and with 
# saturation mixing ratio between lcl and freezing level kg/kg
RLAYER = np.append(MixingRatio[0:lclIndex], satq.magnitude) 

TLAYER = np.append(Temperature[0:lclIndex]+ 273.155, temp_cloud.magnitude)
PLAYER = Pressure[0:HailReleaseIndex+1] * 100.0  # Mean sub and in cloud pressure, hPa back to Pa
HLAYER = Height[0:HailReleaseIndex+1]

# Determine freezing level in cloud
frzlvl = np.squeeze(np.array(np.nanargmin(np.absolute(TLAYER-273.155)))) 
freezing_height = HLAYER[frzlvl] - HLAYER[0]

# Change names/units to what terminl, melt are expecting, and subset to below freezing lvl
zh = Height
zh = zh[0:HailReleaseIndex+1] # height above ground, start at flight level
zh = zh - zh[0]

DENSA = rho[0:HailReleaseIndex+1].magnitude # in cloud density
DENSE = 917. # Hail density [kg m-3]

# Initialize hail tracking arrays
alld = np.ones((numd, numt, 6, 4, 5), dtype=float)*np.nan # all diameters
allz = np.ones((numd, numt, 6, 4, 5), dtype=float)*np.nan # all heights

Up = np.ones((3, numd, 6), dtype=float)*np.nan
Gone = np.ones((3, numd, 6), dtype=float)*np.nan

EndingHeight = np.ones((numd, 6, 6), dtype=float)*np.nan
EndingDiameter = np.ones((numd, 6, 6), dtype=float)*np.nan
EndingTime = np.ones((numd, 6, 6), dtype=float)*np.nan
EndingState = np.empty((numd, 7, 6), dtype=float)*np.nan # surface = nan, melt = 1, up = 2
Count = 0

LCLhailSize_Distribution = np.empty((6, 5, 5, numd), dtype=float)*np.nan
SurfaceHailSize_Distribution = np.empty((6, 5, 5, numd), dtype=float)*np.nan

##### RUN HAILCAST #####
## Set parameterizations ##
# Loop through all options, calculating hail property evolution and plot
# First select on melt option and then loop through all terminal velocity options
# Will also loop through all environmental options
for m in range(0, 4):            
    for t in range(0, 5):                
        print(m, t, MeltNames[m], TerminalNames[t])

        ## Create modified updrafts ##
        Updraft = np.empty((len(HeightProfile), 6), dtype=float)*np.nan
        for c in range (0, 6):
            # Set assumed updraft characteristics
            if c == 0:                
                #print('Constant: -2')
                UpdraftStartSpeed = -2
                UpdraftEndSpeed = -2
            elif c == 1:
                #print('Constant: 2')
                UpdraftStartSpeed = 2
                UpdraftEndSpeed = 2            
            elif c == 2:
                #print('Linear: -2 -> 2')
                UpdraftStartSpeed = -2
                UpdraftEndSpeed = 2
            elif c == 3:
                #print('Linear: 0 -> 8')
                UpdraftStartSpeed = 0
                UpdraftEndSpeed = 8
            elif c == 4: 
                #print('Linear -2 -> Mean')
                UpdraftStartSpeed = -2
                UpdraftEndSpeed = 5 #np.nanmean(VerticalVelocity[VerticalVelocity > 0])
            elif c == 5: 
                #print('Constant: Mean')
                UpdraftStartSpeed = 5 #np.nanmean(VerticalVelocity[VerticalVelocity > 0])
                UpdraftEndSpeed = 5 #np.nanmean(VerticalVelocity[VerticalVelocity > 0])

            # Calculate vertical velocity profile 
            if c == 0 or c ==1:
                Index = np.array(np.nanargmin(np.absolute(np.subtract(HeightProfile, lcl_height))))
                Updraft[Index::, c] = np.ones_like(HeightProfile[Index::])*UpdraftStartSpeed
                Updraft[0:Index, c] = 0
            elif c ==2 or c == 3:
                Coefficients = np.polyfit([0, freezing_height], [UpdraftStartSpeed, UpdraftEndSpeed], 1)
                Line = np.poly1d(Coefficients)
                Updraft[:, c] = Line(HeightProfile)
            elif c == 4:
                if UpdraftEndSpeed <= 0 or np.isnan(UpdraftEndSpeed):
                    UpdraftStartSpeed = 0
                    
                Coefficients = np.polyfit([0, freezing_height], [UpdraftStartSpeed, UpdraftEndSpeed], 1)
                Line = np.poly1d(Coefficients)
                Updraft[:, c] = Line(HeightProfile)
            elif c == 5:
                if UpdraftEndSpeed <= 0 or np.isnan(UpdraftEndSpeed):
                    UpdraftStartSpeed = 0
                    
                Index = np.array(np.nanargmin(np.absolute(np.subtract(HeightProfile, lcl_height))))
                Updraft[Index::, c] = np.ones_like(HeightProfile[Index::])*UpdraftStartSpeed
                Updraft[0:Index, c] = 0

            # Determine initial hail conditions (environment and terminal velocity)
            # Find properties of initial hailstone location, assuming starting at freezing level 
            z = zh[-1] # height above ground
            p = PLAYER[-1]
            d_in = np.copy(d_0)
        
            # Initial terminal velocity calculation
            if t == 0: 
                tv = [terminl_Bohm89X_Re.terminl_bohm89x_re(DENSA[-1], DENSE, d, TLAYER[-1]) for d in d_in] 
            elif t == 1:
                tv = [terminl_H2018V_D.terminl_h2018v_d(d, p) for d in d_in]
            elif t == 2:
                tv = [terminl_H2018X_Re.terminl_h2018x_re(DENSA[-1], DENSE, d, TLAYER[-1]) for d in d_in]
            elif t == 3:
                tv = [terminl_HW2014X_Re.terminl_hw2014x_re(DENSA[-1], DENSE, d, TLAYER[-1]) for d in d_in]   
            elif t == 4: 
                tv = [terminl_RH87X_Re.terminl_rh87x_re(DENSA[-1], DENSE, d, TLAYER[-1]) for d in d_in]
                
            tv = np.array(tv)
            
            # Initialize additional arrays 
            MeltMass_Running = np.zeros((numd), dtype=float)
            raindrop = np.zeros((3, numd, numt), dtype=float)

            # loop over all times
            for n in np.arange(numt): # loop through times  
                #calculate distance hailstone has fallen
                ldepth = tv*secdel # depth fell
                z = z - ldepth # updated height
                    
                d_in[z <= 0] = np.nan
                z[z <= 0] = np.nan

                allz[:, n, c, m, t] = z                    
                if np.all(z<0) or np.all(np.isnan(z)): # Break out of loop if all stones have hit the ground
                    break

                #interpolate new temp, density, pressure at new height
                NEWTK = np.interp(z, zh, TLAYER) 
                NEWP = np.interp(z, zh, PLAYER)
                NEWR = np.interp(z, zh, RLAYER)
                NEWDENSA = np.interp(z, zh, DENSA)

                ## Melt ##
                # Determine if warm enough to melt. If warm enough melt, change mass, determine water fraction
                newd = np.ones_like(d_in)*np.nan
                for d in np.arange(numd):
                    # Determine starting mass
                    Mass_Original = (4/3)*pi*(d_in[d]/2)**3 * DENSE
                        
                    if m == 0: 
                        if NEWTK[d] > 273.155:
                            # Warm enough, melt hail
                            newd[d] = melt_G1969orig.melt_g1969orig(d_in[d], NEWTK[d], NEWP[d], NEWR[d], 
                                                                    ldepth[d], tv[d])
                            # Determine if hail still exists
                            if newd[d] < 1e-4 or np.isnan(newd[d]):
                                # Hail FW
                                newd[d] = np.nan
                                Gone[0, d, c] = d_in[d]
                                Gone[1, d, c] = z[d]
                                Gone[2, d, c] = n
                            else:
                                # Calculate mass change and water fraction
                                Mass_New = (4/3)*pi*(newd[d]/2)**3 * DENSE                        
                                MeltMass_Step] = Mass_Original - Mass_New
                                MeltMass_Running[d] = MeltMass_Running[d] + MeltMass_Step]
                                FW = MeltMass_Running[d]/(Mass_New + MeltMass_Running[d])
                                
                        elif NEWTK[d] <= 273.155:
                            # Too cold, hail remains same size
                            newd[d] = d_in[d]
                        
                    elif m == 1: 
                        if NEWTK[d] > 273.155:
                            # Warm enough, melt hail
                            newd[d] = melt_RH87.melt_rh87orig(d_in[d], NEWTK[d], NEWP[d], NEWR[d], tv[d], secdel, 
                                                          DENSE, DENSA)
                            # Determine if hail still exists
                            if newd[d] < 1e-4 or np.isnan(newd[d]):
                                # Hail gone
                                newd[d] = np.nan
                                Gone[0, d, c] = d_in[d]
                                Gone[1, d, c] = z[d]
                                Gone[2, d, c] = n
                            else:
                                # Calculate mass change and water fraction
                                Mass_New = (4/3)*pi*(newd[d]/2)**3 * DENSE                        
                                MeltMass_Step] = Mass_Original - Mass_New
                                MeltMass_Running[d] = MeltMass_Running[d] + MeltMass_Step]
                                FW = MeltMass_Running[d]/(Mass_New + MeltMass_Running[d])
                                
                        elif NEWTK[d] <= 273.155:
                            # Too cold, hail remains same size
                            newd[d] = d_in[d]
                            
                    elif m == 2: 
                        if NEWTK[d] > 273.155:
                            # Warm enough, melt hail
                            newd[d] = melt_WC2020.melt_wc2020short(d_in[d], NEWTK[d], NEWP[d], NEWR[d], tv[d], 
                                                              secdel, DENSE, DENSA)
                            # Determine if hail still exists
                            if newd[d] < 1e-4 or np.isnan(newd[d]):
                                # Hail gone
                                newd[d] = np.nan
                                Gone[0, d, c] = d_in[d]
                                Gone[1, d, c] = z[d]
                                Gone[2, d, c] = n
                            else:
                                # Calculate mass change and water fraction
                                Mass_New = (4/3)*pi*(newd[d]/2)**3 * DENSE                        
                                MeltMass_Step] = Mass_Original - Mass_New
                                MeltMass_Running[d] = MeltMass_Running[d] + MeltMass_Step]
                                FW = MeltMass_Running[d]/(Mass_New + MeltMass_Running[d])
                                
                        elif NEWTK[d] <= 273.155:
                            # Too cold, hail remains same size
                            newd[d] = d_in[d]
                            
                    elif m == 3: 
                        if NEWTK[d] > 273.155:
                            # Warm enough, melt hail
                            newd[d] = melt_ZL1994.melt_zl1994sphere(d_in[d], NEWTK[d], NEWP[d], NEWR[d], tv[d], 
                                                              secdel, DENSE, DENSA)
                            # Determine if hail still exists
                            if newd[d] < 1e-4 or np.isnan(newd[d]):
                                # Hail gone
                                newd[d] = np.nan
                                Gone[0, d, c] = d_in[d]
                                Gone[1, d, c] = z[d]
                                Gone[2, d, c] = n
                            else:
                                # Calculate mass change and water fraction
                                Mass_New = (4/3)*pi*(newd[d]/2)**3 * DENSE                        
                                MeltMass_Step] = Mass_Original - Mass_New
                                MeltMass_Running[d] = MeltMass_Running[d] + MeltMass_Step]
                                FW = MeltMass_Running[d]/(Mass_New + MeltMass_Running[d])
                                
                        elif NEWTK[d] <= 273.155:
                            # Too cold, hail remains same size
                            newd[d] = d_in[d] 

                    ## Shedding ##
                    if SheddingFlag == True:
                        # Only shed if hail exists and above freezing
                        if newd[d] > 0 or ~np.isnan(newd[d]) and NEWTK[d] > 273.15:
        
                            # If the mass of the melted water on the hail is above a critical limit, shed water
                            if MeltMass_Running[d] > CRIT:
                                shedwater_mass[d, n] = MeltMass_Running[d] - CRIT
                                #print('shedwater', shedwater_mass, d_in[d])
        
                                # Calculate size and number cloud drops shed (Based on Fig 1 in Mansell 2020 JAS)
                                if newd[d] >= 4.5*1e-3 and newd[d] < 8*1e-3:
                                    radius = (4.5*1e-3)/2
                                    DropMass = (4/3)*pi*radius*radius*radius*997
                                    raindrop[0, d, n] = raindrop[0, d, n] + np.floor(np.divide(shedwater_mass[d, n], DropMass))*num_each_diameter[d]
                                    
                                elif newd[d] >= 8*1e-3 and newd[d] < 9.5*1e-3:
                                    radius = (3*1e-3)/2
                                    DropMass = (4/3)*pi*radius*radius*radius*997
                                    raindrop[1, d, n] = raindrop[1, d, n] + np.floor(np.divide(shedwater_mass[d, n], DropMass))*num_each_diameter[d]

                                elif newd[d] >= 9.5*1e-3:
                                    radius = (1.5*1e-3)/2
                                    DropMass = (4/3)*pi*radius*radius*radius*997
                                    raindrop[2, d, n] = raindrop[2, d, n] + np.floor(np.divide(shedwater_mass[d, n], DropMass))*num_each_diameter[d]

                                # Determine change in running mass melt and FW after shedding
                                MeltMass_Running[d] = np.copy(CRIT)
                                FW = CRIT/(Mass_New + CRIT)
                                    
                                # Restrict water fraction to between 0 -1
                                if FW >= 1:
                                    newd[d] = np.nan
                                    Gone[0, d, c] = d_in[d]
                                    Gone[1, d, c] = z[d]
                                    Gone[2, d, c] = n
                        else:
                            # Conditions are not supportive for possible shedding
                            newd[d] = d_in[d]
                    
                # Save melt, raindrop, and hail diameter data. 
                alld[:, n, c, m, t] = np.copy(newd) # Save diameters
                
                # Set hail diameters for next time step
                d_in = np.array(newd) # Update diameters

                if n < numt-1:
                    raindrop[:, :, n+1] = np.copy(raindrop[:, :, n])

                ## Terminal velocity ##
                # These are terminal velocities for the "new hail"
                # Negative tv indicates that the hail is swept upward
                if t == 0: 
                    tv = [terminl_Bohm89X_Re.terminl_bohm89x_re(NEWDENSA[d], DENSE, newd[d], NEWTK[d]) for d in np.arange(numd)] 
                elif t == 1:
                    tv = [terminl_H2018V_D.terminl_h2018v_d(newd[d], NEWP[d]) for d in np.arange(numd)]
                elif t == 2:
                    tv = [terminl_H2018X_Re.terminl_h2018x_re(NEWDENSA[d], DENSE, newd[d], NEWTK[d]) for d in np.arange(numd)] 
                elif t == 3:
                    tv = [terminl_HW2014X_Re.terminl_hw2014x_re(NEWDENSA[d], DENSE, newd[d], NEWTK[d]) for d in np.arange(numd)]    
                elif t == 4: 
                    tv = [terminl_RH87X_Re.terminl_rh87x_re(NEWDENSA[d], DENSE, newd[d], NEWTK[d]) for d in np.arange(numd)]
                tv = np.array(tv)

                # Correct terminal velocity for impact of updraft velocity
                for d in range(0, numd):
                    if ~np.isnan(newd[d]) and ~np.isnan(newd[d]):
                        # Determine where the hail is in the height profile
                        Index = np.squeeze(np.array(np.nanargmin(np.absolute(np.subtract(HeightProfile, 
                                                                                         np.ones_like(HeightProfile)
                                                                                         *(z[d]))))))
                        if Updraft[Index, c] < tv[d]:
                            # Terminal velocity is stronger than updraft. 
                            # Reduce terminal velocity by removing updraft component
                            tv[d] = tv[d] - Updraft[Index, c]
                        else:
                            # Terminal velocity is weaker than updraft. Hail is swept up.
                            # Stop tracking hail 
                            tv[d] = np.nan
                            Up[0, d, c] = d_in[d]
                            Up[1, d, c] = z[d]
                            Up[2, d, c] = n

                # Remove data if hail melts or is swept upward
                d_in[~np.isnan(Gone[0, :, c])] = np.nan
                d_in[~np.isnan(Up[0, :, c])] = np.nan
                z[~np.isnan(Gone[0, :, c])] = np.nan
                z[~np.isnan(Up[0, :, c])] = np.nan
                
        ############ Plot Results #          
        # Plot time series
        # Height and diameter
        for i in np.arange(numd):
            fig = plt.figure(figsize=(8, 8))
            plt.suptitle('Terminal: ' + TerminalNames[t] + ', Melt: ' + MeltNames[m] + ' Hail Stone Evolution \n Initial Diameter: ' 
                         + str(np.round(d_0[i]*1.e3, 2)) + ' mm',size=16, y=0.98 )

            ax1 = plt.subplot(211)
            ax1.set_title('Height vs Time', size=14)
            for c in range(0, 6):
                ax1.plot(np.arange(0, numt, 1), allz[i, :, c, m, t]/1000, linewidth=2, color=PlotColors[c], 
                             label=PlotLabels[c])
            for c in range(0, 6):
                ax1.scatter(Gone[2, i, c], Gone[1, i, c, m, t]/1000, c='black', marker='o', s=20)
                ax1.scatter(Up[2,  i, c], Up[1, i, c, m, t]/1000, c='black', marker='x', s=20)
            ax1.set_ylim(bottom=0)
            ax1.set_ylabel('Height [km]', size=14)
            #ax1.set_xlim(0, 900)
            ax1.tick_params(axis='x', labelbottom = False)
            ax1.tick_params(axis='both', labelsize=12)
            ax1.grid(True)

            ax2 = plt.subplot(212)
            ax2.set_title('Diameter vs Time', size=14)
            for c in range(0, 6):
                ax2.plot(np.arange(0, numt, 1), alld[i, :, c, m, t]*1.E3, linewidth=2, color=PlotColors[c], 
                             label=PlotLabels[c])
            for c in range(0, 6):
                ax2.scatter(Gone[2, i, c], Gone[0, i, c, m, t]*1.E3, c='black', marker='o', s=20)
                ax2.scatter(Up[2,  i, c], Up[0, i, c, m, t]*1.E3, c='black', marker='x', s=20)
            ax2.set_ylim(0, np.nanmax(alld[i, :, :, m, t])*1.E3)
            ax2.set_ylabel('Diameter [mm]', size=14)
            #ax2.set_xlim(0, 900)
            ax2.set_xlabel('Time Step', size=14)
            #ax2.tick_params(axis='x', labelbottom = False)
            ax2.tick_params(axis='both', labelsize=12)
            ax2.grid(True)
            
            plt.tight_layout()
            plt.savefig(PlotDirectory + TerminalNames[t]  + '-' + MeltNames[m] + '_TimeSeries_' + str(np.round(d_0[i]*1.e3, 2)) + 'mm_VV.png', 
                        dpi=100, format='png')
            plt.close()

        # Plot updrafts
        if m == 0 and t == 0:
            plt.figure(figsize=(4, 5))
            plt.title('Assumed Vertical Velocity Profiles')
            for c in range (0, 6):
                plt.plot(Updraft[:, c], HeightProfile/1000, color=PlotColors[c], label=PlotLabels[c], linewidth=3)
            plt.legend(fontsize=10)
            plt.ylim(0, 6)
            ax = plt.gca()
            yLim = ax.get_ylim()
            xLim = ax.get_xlim()
            plt.plot([0, 0], [yLim[0], yLim[1]], color='dimgray')
            plt.plot([xLim[0], xLim[1]], [freezing_height/1000, freezing_height/1000], color='dimgray', 
                     linestyle='--')
            plt.ylim(0, 5)
            plt.ylabel('Height [km]')
            plt.xlim(-2.5, 10)
            #plt.xlim(xLim[0], xLim[1])
            plt.xlabel('Vertical Velocity [m/s]')
            #plt.tick_params(labelsize=14)
            plt.grid(True)
            plt.savefig(PlotDirectory + 'Updrafts.png', dpi=100, format='png')
            plt.close()

# Final hail Variability plots
RawEndingDiameter = np.copy(EndingDiameter)
RawEndingHeight = np.copy(EndingHeight)
RawEndingTime = np.copy(EndingTime)
for i in range(0, 6):
    for c in range(0, 6):
        EndingDiameter[:, c, i] = EndingDiameter[:, c, i] - RawEndingDiameter[:, 5, 1]
        EndingHeight[:, c, i] = EndingHeight[:, c, i] - RawEndingHeight[:, 5, 1]
        EndingTime[:, c, i] = EndingTime[:, c, i] - RawEndingTime[:, 5, 1]

for i in np.arange(1, 13, 4): #np.arange(1, 13, 4):
    #ColorMax = np.nanmax([np.nanpercentile(EndingDiameter[np.arange(5, 13, 4), :, :], 99.75), 
    #                      np.absolute(np.nanpercentile(EndingDiameter[np.arange(5, 13, 4), :, :], 0.25))])
    ColorMax = 18
    plt.figure(figsize=(12, 5))
    plt.title('Ending Size for Hail starting at ' + str(np.round(d_0[i]*1e3, 2)) + ' mm Relative to RH87', y=1.01)
    #C = plt.pcolormesh(EndingDiameter[i, :, :]*1e3, vmin=-1*ColorMax*1e3, vmax=ColorMax*1e3, cmap=cmap_diff)  
    C = plt.pcolormesh(EndingDiameter[i, :, :]*1e3, vmin=-1*ColorMax, vmax=ColorMax, cmap=cmap_diff) 
    plt.colorbar(C, label='Hail Size [mm]', pad=0.01)
    plt.plot([1, 1], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([2, 2], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([3, 3], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([4, 4], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([5, 5], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([6, 6], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [1, 1], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [2, 2], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [3, 3], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [4, 4], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [5, 5], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [6, 6], color='dimgray', alpha=0.5)
    plt.xticks(np.arange(0.5, 6.5, 1), [MeltNames[1] + ' \n ' + TerminalNames[1], MeltNames[1] + ' \n ' + TerminalNames[4],
                                        MeltNames[2] + ' \n ' + TerminalNames[1], MeltNames[2] + ' \n ' + TerminalNames[4],  
                                        MeltNames[3] + ' \n ' + TerminalNames[1], MeltNames[3] + ' \n ' + TerminalNames[4]], fontsize=10)
    plt.xlim(0, 6)
    plt.xlabel('Parameterization Combinations')
    plt.yticks(np.arange(0.5, 6.5), PlotLabels, fontsize=10)
    plt.ylim(0, 6)
    plt.ylabel('Environment Conditions')
    plt.tight_layout()
    plt.savefig(PlotDirectory + 'EndDiameterHeatMap_' +str(np.round(d_0[i]*1e3, 2)) + 'mm_VV.png', dpi=100, format='png')
    plt.close()

    #ColorMax = np.nanmax([np.nanpercentile(EndingHeight[np.arange(5, 13, 4), :, :], 99), 
    #                      np.absolute(np.nanpercentile(EndingHeight[np.arange(5, 13, 4), :, :], 1))])
    ColorMax = 7000
    plt.figure(figsize=(12, 5))
    plt.title('Ending Height for Hail starting at ' + 
              str(np.round(d_0[i]*1e3, 2)) + ' mm Relative to RH87', y=1.01)
    C = plt.pcolormesh(EndingHeight[i, :, :]/1000, vmin=-1*ColorMax/1000, vmax=ColorMax/1000, cmap=cmap_diff) 
    plt.colorbar(C, label='Height [km]', pad=0.01)
    plt.plot([1, 1], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([2, 2], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([3, 3], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([4, 4], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([5, 5], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([6, 6], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [1, 1], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [2, 2], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [3, 3], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [4, 4], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [5, 5], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [6, 6], color='dimgray', alpha=0.5)
    plt.xticks(np.arange(0.5, 6.5, 1), [MeltNames[1] + ' \n ' + TerminalNames[1], MeltNames[1] + ' \n ' + TerminalNames[4],
                                        MeltNames[2] + ' \n ' + TerminalNames[1], MeltNames[2] + ' \n ' + TerminalNames[4],  
                                        MeltNames[3] + ' \n ' + TerminalNames[1], MeltNames[3] + ' \n ' + TerminalNames[4]], fontsize=10)
    plt.xlim(0, 6)
    plt.xlabel('Parameterization Combinations')
    plt.yticks(np.arange(0.5, 6.5), PlotLabels, fontsize=10)
    plt.ylim(0, 6)
    plt.ylabel('Environment Conditions')
    plt.tight_layout()
    plt.savefig(PlotDirectory + 'EndHeightHeatMap_' +str(np.round(d_0[i]*1e3, 2)) + 'mm_VV.png', dpi=100, format='png')
    plt.close()

    #ColorMax = np.nanmax([np.nanpercentile(EndingTime[np.arange(5, 13, 4), :, :], 99), 
    #                      np.absolute(np.nanpercentile(EndingTime[np.arange(5, 13, 4), :, :], 1))])
    ColorMax = 1900
    plt.figure(figsize=(12, 5))
    plt.title('Ending Time for Hail starting at ' + 
              str(np.round(d_0[i]*1e3, 2)) + ' mm Relative to RH87', y=1.01)
    C = plt.pcolormesh(EndingTime[i, :, :], vmin=-1*ColorMax, vmax=ColorMax, cmap=cmap_diff) 
    plt.colorbar(C, label='Time Step [s]', pad=0.01)
    plt.plot([1, 1], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([2, 2], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([3, 3], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([4, 4], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([5, 5], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([6, 6], [0, 8], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [1, 1], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [2, 2], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [3, 3], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [4, 4], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [5, 5], color='dimgray', alpha=0.5)
    plt.plot([0, 10], [6, 6], color='dimgray', alpha=0.5)
    plt.xticks(np.arange(0.5, 6.5, 1), [MeltNames[1] + ' \n ' + TerminalNames[1], MeltNames[1] + ' \n ' + TerminalNames[4],
                                        MeltNames[2] + ' \n ' + TerminalNames[1], MeltNames[2] + ' \n ' + TerminalNames[4],  
                                        MeltNames[3] + ' \n ' + TerminalNames[1], MeltNames[3] + ' \n ' + TerminalNames[4]], fontsize=10)
    plt.xlim(0, 6)
    plt.xlabel('Parameterization Combinations')
    plt.yticks(np.arange(0.5, 6.5), PlotLabels, fontsize=10)
    plt.ylim(0, 6)
    plt.ylabel('Environment Conditions')
    plt.tight_layout()
    plt.savefig(PlotDirectory + 'EndTimeHeatMap_' +str(np.round(d_0[i]*1e3, 2)) + 'mm_VV.png', dpi=100, format='png')
    plt.close()
    

761
/Users/hvagasky/Data/PMM_Hail/Soundings/mobile1.steps.qc_soundings/D200006292355.st1QC.cls
0 0 mG1969orig tBohm89xre
0 1 mG1969orig tH2018vd
0 2 mG1969orig tH2018xre


/var/folders/cb/zhyy0qr14kg35txh2x2n3hvm0002n2/T/ipykernel_17325/221433411.py:159: RuntimeWarning: invalid value encountered in multiply
  Updraft = np.empty((len(HeightProfile), 6), dtype=float)*np.nan


0 3 mG1969orig tHW2014xre
0 4 mG1969orig tRH87xre
1 0 mRH87 tBohm89xre
1 1 mRH87 tH2018vd
1 2 mRH87 tH2018xre
1 3 mRH87 tHW2014xre
1 4 mRH87 tRH87xre
2 0 mWC2020 tBohm89xre
2 1 mWC2020 tH2018vd
2 2 mWC2020 tH2018xre
2 3 mWC2020 tHW2014xre
2 4 mWC2020 tRH87xre
3 0 mZL1994 tBohm89xre
3 1 mZL1994 tH2018vd
3 2 mZL1994 tH2018xre
3 3 mZL1994 tHW2014xre
3 4 mZL1994 tRH87xre
